In [4]:
import json
import boto3
from io import BytesIO
import requests
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()
access_key = os.getenv("ACCESS_KEY")
secret_key = os.getenv("SECRET_KEY")

# Initialize S3 client
s3_client = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)
bucket_name = 'learnsnowflakedbt-heitor'

BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"

deputados = pd.read_csv('deputados.csv', sep=';')

In [10]:
from tqdm import tqdm

# Ordenar o DataFrame deputados por idLegislaturaInicial em ordem decrescente
deputados_sorted = deputados[deputados["idLegislaturaInicial"] > 0].sort_values(by="idLegislaturaInicial", ascending=False)
# Lista para armazenar os dados dos históricos
historicos = []

# Loop pelos URLs dos deputados com tqdm para monitorar o progresso
for url in tqdm(deputados_sorted['uri'], desc="Processando históricos dos deputados"):
    deputado_id = url.split('/')[-1]  # Extrair o ID do deputado do URL
    endpoint = f"{BASE_URL}/deputados/{deputado_id}/historico"
    
    # Fazer a requisição para obter o histórico do deputado
    response = requests.get(endpoint).json()
    if 'dados' in response:
        for historico in response['dados']:
            historico['deputado_id'] = deputado_id  # Adicionar o ID do deputado ao histórico
            historicos.append(historico)

# Converter a lista de históricos em um DataFrame
df_historicos = pd.DataFrame(historicos)

# Exibir o DataFrame resultante
df_historicos


Processando históricos dos deputados: 100%|██████████| 7865/7865 [27:55<00:00,  4.69it/s]  


,id,uri,nome,nomeEleitoral,siglaPartido,uriPartido,siglaUf,idLegislatura,email,urlFoto,dataHora,situacao,condicaoEleitoral,descricaoStatus,deputado_id
0,220603,https://dadosabertos.camara.leg.br/api/v2/depu...,Dani Cunha,Dani Cunha,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,57,None,https://www.camara.leg.br/internet/deputado/ba...,2023-02-01T00:00,Exercício,Titular,Nome no início da legislatura / Partido no iní...,220603
1,220603,https://dadosabertos.camara.leg.br/api/v2/depu...,Dani Cunha,Dani Cunha,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,57,None,https://www.camara.leg.br/internet/deputado/ba...,2023-02-01T12:05,Exercício,Titular,Entrada - Posse de Eleito Titular - Posse na S...,220603
2,220629,https://dadosabertos.camara.leg.br/api/v2/depu...,Dandara,Dandara,PT,https://dadosabertos.camara.leg.br/api/v2/part...,MG,57,None,https://www.camara.leg.br/internet/deputado/ba...,2023-02-01T00:00,Exercício,Titular,Nome no início da legislatura / Partido no iní...,220629
3,220629,https://dadosabertos.camara.leg.br/api/v2/depu...,Dandara,Dandara,PT,https://dadosabertos.camara.leg.br/api/v2/part...,MG,57,None,https://www.camara.leg.br/internet/deputado/ba...,2023-02-01T12:05,Exercício,Titular,Entrada - Posse de Eleito Titular - Posse na S...,220629
4,220577,https://dadosabertos.camara.leg.br/api/v2/depu...,Augusto Pupio,Augusto Pupio,MDB,https://dadosabertos.camara.leg.br/api/v2/part...,AP,57,None,https://www.camara.leg.br/internet/deputado/ba...,2023-02-01T00:00,Exercício,Titular,Nome no início da legislatura / Partido no iní...,220577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54153,183,https://dadosabertos.camara.leg.br/api/v2/depu...,VICENTE DA FONSECA,VICENTE DA FONSECA,None,None,SP,1,None,https://www.camara.leg.br/internet/deputado/ba...,1827-04-26T00:00,Exercício,Titular,Primeira posse na legislatura (dados legados),183
54154,183,https://dadosabertos.camara.leg.br/api/v2/depu...,VICENTE DA FONSECA,VICENTE DA FONSECA,None,None,SP,1,None,https://www.camara.leg.br/internet/deputado/ba...,1830-04-24T23:59,None,Titular,Situação e condição ao fim da legislatura (dad...,183
54155,170,https://dadosabertos.camara.leg.br/api/v2/depu...,XAVIER FERREIRA,XAVIER FERREIRA,None,None,RS,1,None,https://www.camara.leg.br/internet/deputado/ba...,1826-04-29T00:00,None,None,Nome no início da legislatura,170
54156,170,https://dadosabertos.camara.leg.br/api/v2/depu...,XAVIER FERREIRA,XAVIER FERREIRA,None,None,RS,1,None,https://www.camara.leg.br/internet/deputado/ba...,1826-05-17T00:00,Exercício,Titular,Primeira posse na legislatura (dados legados),170


In [11]:
len(df_historicos)

54158

In [12]:
# Salvar o DataFrame df_historicos em um arquivo Parquet
parquet_buffer = BytesIO()
df_historicos.to_parquet(parquet_buffer, index=False)

# Enviar o arquivo Parquet para o bucket S3
s3_client.put_object(
    Bucket=bucket_name,
    Key="camara/deputados/parquet/historicos-deputados.parquet",
    Body=parquet_buffer.getvalue()
)


{'ResponseMetadata': {'RequestId': 'A92KVRH5APXQYZN4',
  'HostId': 'NR95uEZR3ZU982KFa6zRPMyBv21Z6ZCpI7N1DvjPDJHa+Xq/eewVPq3qU122CnCVDv1OKa7O3hg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'NR95uEZR3ZU982KFa6zRPMyBv21Z6ZCpI7N1DvjPDJHa+Xq/eewVPq3qU122CnCVDv1OKa7O3hg=',
   'x-amz-request-id': 'A92KVRH5APXQYZN4',
   'date': 'Wed, 23 Jul 2025 23:55:40 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"bd9682fa1e382dd90b77f35603283389"',
   'x-amz-checksum-crc32': '45RZVQ==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"bd9682fa1e382dd90b77f35603283389"',
 'ChecksumCRC32': '45RZVQ==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

In [13]:
details = []

for url in tqdm(deputados_sorted['uri'], desc="Processando históricos dos deputados"):

    
    # Fazer a requisição para obter o histórico do deputado
    response = requests.get(url).json()
    if 'dados' in response:
        detail = response['dados']
        details.append(detail)





Processando históricos dos deputados: 100%|██████████| 7865/7865 [27:47<00:00,  4.72it/s]   


In [17]:
array_str = json.loads(open('response_modec_mexico_aso', 'r', encoding='utf-8').read())
len(array_str)

1319

In [19]:
total_fitness_certificates = sum(len(item.get('fitness_certificates', [])) for item in array_str)
total_fitness_certificates

2075

In [14]:
df_details = pd.DataFrame(details)


# Salvar o DataFrame df_historicos em um arquivo Parquet
parquet_buffer = BytesIO()
df_details.to_parquet(parquet_buffer, index=False)

# Enviar o arquivo Parquet para o bucket S3
s3_client.put_object(
    Bucket=bucket_name,
    Key="camara/deputados/parquet/deputados_detail.parquet",
    Body=parquet_buffer.getvalue()
)


{'ResponseMetadata': {'RequestId': 'DTAS72PS0J5RPTP5',
  'HostId': 'GkXGzGpHa9MGClhw98XEQ2D4d/kxtkJMg9wNr99b7+VIXFTTrEUNclO9Bp1wZJJrOIlqhfTo4nA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'GkXGzGpHa9MGClhw98XEQ2D4d/kxtkJMg9wNr99b7+VIXFTTrEUNclO9Bp1wZJJrOIlqhfTo4nA=',
   'x-amz-request-id': 'DTAS72PS0J5RPTP5',
   'date': 'Thu, 24 Jul 2025 00:23:34 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"25cad05b65bcfbfd822b2b72b22f241e"',
   'x-amz-checksum-crc32': 'b3ED9A==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"25cad05b65bcfbfd822b2b72b22f241e"',
 'ChecksumCRC32': 'b3ED9A==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}